In [36]:
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [37]:
df = pd.read_csv('train.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [38]:
df.shape

(27480, 4)

In [39]:
# Filtre le DataFrame pr éliminer les lignes où 'neutral' apparait : 
df_filtered = df[df['sentiment'] != 'neutral']

df_filtered

,textID,text,selected_text,sentiment
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
...,...,...,...,...
27474,b78ec00df5,enjoy ur night,enjoy,positive
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive


In [40]:
df_filtered.shape

(16363, 4)

In [41]:
# Calcule la fréqce des sentimts :
sentiment = df_filtered['sentiment'].value_counts()

# Calcule le % de chaq sentimt :
sentiment_pourcent = (sentiment / sentiment.sum()) * 100

sentiment_pourcent

sentiment
positive    52.447595
negative    47.552405
Name: count, dtype: float64

In [42]:
# Copie la colonne 'text' ds 1 Série X et la colonne 'sentimt' ds 1 Série y :
X = df_filtered['text']
y = df_filtered['sentiment']

# Appliq 1 train-test split avec random_state=32 et train_size=0.75 :
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, train_size=0.75, random_state=32)

# Affiche les résultats :
print("X_train:")
print(X_train)
print("\nX_test:")
print(X_test)
print("\ny_train:")
print(y_train)
print("\ny_test:")
print(y_test)

X_train:
9186                                  Wish I was @ wembley
12345    and because I wasn`t having a bad enough time ...
20086    _Perth Awesome...love top gear..whos your favo...
1674     totally forgot she had soccer today. ugh  toda...
21750                                 you are very welcome
                               ...                        
18649             so...i really want to be home right now.
16055     I realised when I got home that I left mine i...
27303    _Kay2 welcome to Twitter babes! I know what yo...
16444    i`m doin my tweets on my phone so i have on cl...
18392                        still in love with marky mark
Name: text, Length: 12272, dtype: object

X_test:
5680      - no,  is buttfuck stupid. I`m just silly and...
7661      get better omg i still dont believe that i di...
2670     HollowbabesHere comes the utter shite #bgt <I ...
5020      Thank You Clayton. Going to my favorite Greek...
26962     I`m watching it at the moment  -sighs- and st.

In [43]:
# Créer 1 mdèle CountVectorizer :
vectorizer = CountVectorizer()

# Entraîne le mdèle sur X_train et créer la matrice de caractéristiq X_train_CV :
X_train_CV = vectorizer.fit_transform(X_train)

# Créer la matrice de caractéristiq X_test_CV en utilisant le même mdèle sans réentraîner :
X_test_CV = vectorizer.transform(X_test)

# Affiche la forme de la matrice X_train_CV et X_test_CV :
print("Shape de X_train_CV:", X_train_CV.shape)
print("Shape de X_test_CV:", X_test_CV.shape)

Shape de X_train_CV: (12272, 15806)
Shape de X_test_CV: (4091, 15806)


In [45]:
# Initialise la régress° logistiq :
logreg = LogisticRegression()

# Entraîne le mdèle de régress° logistiq sur l'ensble d'entraînemt :
logreg.fit(X_train_CV, y_train)

# Prédis les résultats pr l'ensble d'entraînemt :
y_train_pred = logreg.predict(X_train_CV)

# Prédis les résultats pr l'ensble de test :
y_test_pred = logreg.predict(X_test_CV)

# Calcule l'exactitude pr l'ensble d'entraînemt :
train_accuracy = accuracy_score(y_train, y_train_pred)

# Calcule l'exactitude pr l'ensble de test :
test_accuracy = accuracy_score(y_test, y_test_pred)

# Affiche les résultats :
print("Exactitude sur l'ensble d'entraînemt:", train_accuracy)
print("Exactitude sur l'ensble de test:", test_accuracy)

Exactitude sur l'ensble d'entraînemt: 0.9659387222946545
Exactitude sur l'ensble de test: 0.8772916157418724


In [47]:
# Identifie les faux positifs et faux négatifs :
faux_positifs = []
faux_negatifs = []

# Parcrs les prédict° et les vleurs réelles :
for i in range(len(y_test)):
    if y_test_pred[i] == 'positive' and y_test.iloc[i] != 'positive':
        faux_positifs.append((X_test.iloc[i], y_test.iloc[i], y_test_pred[i]))
    elif y_test_pred[i] == 'negative' and y_test.iloc[i] != 'negative':
        faux_negatifs.append((X_test.iloc[i], y_test.iloc[i], y_test_pred[i]))

# Affiche les 10 premiers faux positifs ou faux négatifs :
# Combine faux positifs et faux négatifs
faux_mal_predits = faux_positifs[:5] + faux_negatifs[:5]  


# Affiche les 10 tweets mal prédits :
for tweet, true_label, pred_label in faux_mal_predits:
    print(f"Tweet: {tweet}")
    print(f"Label réel: {true_label}, Label prédit: {pred_label}")
    print("-" * 80)

Tweet: HollowbabesHere comes the utter shite #bgt <I completely agree
Label réel: negative, Label prédit: positive
--------------------------------------------------------------------------------
Tweet:  SUFFICATION NO BREATHING. It`s okay. There`ll be more. You`re invited to mine, but I can`t promise fun times.  *Jinx
Label réel: negative, Label prédit: positive
--------------------------------------------------------------------------------
Tweet: I love music so much that i`ve gone through pain to play :S my sides of my fingers now are peeling and have blisters from playing so much
Label réel: negative, Label prédit: positive
--------------------------------------------------------------------------------
Tweet:  have an amazing time with your mommas tomorrow! Show them how much they mean to you  Whatever you do they will love it
Label réel: negative, Label prédit: positive
--------------------------------------------------------------------------------
Tweet: Watching 1971 edition 

In [ ]:
# Il se pt que l'on prédise mieux q le modèle dès lors s'il y à la présence d'émojis où de mots ambigus propres aux résx socx.